In [2]:
# Importações de bibliotecas padrão
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Importações de bibliotecas do scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, Lasso
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics.pairwise import cosine_similarity

# Importações de bibliotecas do NLTK
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

# Importações do LIME
import lime
import lime.lime_text
from lime.lime_text import LimeTextExplainer

# Importações do SciPy
from scipy.sparse import csr_matrix

# Importações de funções personalizadas (certifique-se de que os arquivos lime_functions.py e submodular.py estejam no diretório atual)
import lime_functions
import submodular

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Isas_\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:

def sep( n: int = 120, sep: str = "="):
    print(sep*n)

def new_section(string: str, n: int = 120, sep: str = "="):
    m = len(string)
    if n < m:
        n = m + 6
    padding = (n-m+1)//2
    print(sep*padding + " " + string + " " + sep*padding)

def subsection(string: str, n: int = 120, sep: str = "-"):
    print()
    print(string, end="")
    print(sep*n)

In [4]:
new_section("Definindo sentenças para análise")
# Exemplo de conjunto de dados de resenhas
dados = pd.read_csv('dados.csv', sep=';')

# Separar as features e o alvo
X = dados['review']
y = dados['sentimentos']

# Escolher uma instância negativa para fazer a previsão
instance_index_neg = 15
instance_neg = X.iloc[instance_index_neg]
instance_label_neg = y.iloc[instance_index_neg]
print('Instância negativa:', instance_neg)
print('Rótulo negativa:', instance_label_neg)
sep(sep="-")
# Escolher uma instância negativa para fazer a previsão
instance_index_pos = 0
instance_pos = X.iloc[instance_index_pos]
instance_label_pos = y.iloc[instance_index_pos]
print('Instância positiva:', instance_pos)
print('Rótulo positiva:', instance_label_pos)
sep()

============================================ Definindo sentenças para análise ============================================
Instância negativa: Nunca vi algo tão ruim na minha vida. A produção foi amadora e a história, entediante.
Rótulo negativa: 0
------------------------------------------------------------------------------------------------------------------------
Instância positiva: Este filme foi absolutamente incrível! A trama é envolvente e as atuações foram dignas de um Oscar.
Rótulo positiva: 1


In [5]:
# Deixar stopwords
vectorizer = TfidfVectorizer()
X_vectorized = vectorizer.fit_transform(X)

# Dividir os dados em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X_vectorized, y, test_size=0.2, random_state=42)

In [6]:
# Retirar stop-words
stop_words = stopwords.words('portuguese')
X_sw = X.apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

# Vetorização do texto
vectorizer_sw = TfidfVectorizer()
X_vectorized_sw = vectorizer_sw.fit_transform(X_sw)

# Dividir os dados em conjuntos de treino e teste
X_train_sw, X_test_sw, y_train, y_test = train_test_split(X_vectorized_sw, y, test_size=0.2, random_state=42)

In [7]:
# Testes, explorando X_vectorized
new_section("Testes, explorando X_vectorized")
subsection("X_vectorized.shape")
print(X_vectorized.shape)
subsection("X_vectorized[instance_index_neg].indices")
print(X_vectorized[instance_index_neg].indices)
subsection("X_vectorized[instance_index_pos].tocoo().data")
print(X_vectorized[instance_index_pos].tocoo().data)
#subsection("X_vectorized[instance_index_pos].tocoo()")
#print(X_vectorized[instance_index_pos].tocoo())
#subsection("X_vectorized[instance_index_pos].tocoo().col")
#print(X_vectorized[instance_index_pos].tocoo().col)
#subsection("X_vectorized[instance_index_pos][0]")
#print(X_vectorized[instance_index_pos][0]) # Igual a X_vectorized[instance_index_pos].tocoo()
sep()

============================================= Testes, explorando X_vectorized =============================================

X_vectorized.shape------------------------------------------------------------------------------------------------------------------------
(98, 256)

X_vectorized[instance_index_neg].indices------------------------------------------------------------------------------------------------------------------------
[118 126 218 174 250   5 245 164 158 252 200   7  89]

X_vectorized[instance_index_pos].tocoo().data------------------------------------------------------------------------------------------------------------------------
[0.26188687 0.14662237 0.18251715 0.3096147  0.2267131  0.25127256
 0.28980586 0.28980586 0.28980586 0.33753369 0.33753369 0.18624005
 0.1943502  0.33753369]


In [8]:
# Testes, explorando vectorizer
new_section("Testes, explorando vectorizer")
subsection("vectorizer.get_feature_names_out()[0]")
print(vectorizer.get_feature_names_out()[0])
subsection("vectorizer.get_feature_names_out()[1]")
print(vectorizer.get_feature_names_out()[1])
subsection("vectorizer.get_feature_names_out()[2]")
print(vectorizer.get_feature_names_out()[2])
subsection("vectorizer.get_feature_names_out()")
print(vectorizer.get_feature_names_out())
sep()


============================================== Testes, explorando vectorizer ==============================================

vectorizer.get_feature_names_out()[0]------------------------------------------------------------------------------------------------------------------------
absolutamente

vectorizer.get_feature_names_out()[1]------------------------------------------------------------------------------------------------------------------------
achei

vectorizer.get_feature_names_out()[2]------------------------------------------------------------------------------------------------------------------------
acrescentaram

vectorizer.get_feature_names_out()------------------------------------------------------------------------------------------------------------------------
['absolutamente' 'achei' 'acrescentaram' 'adorei' 'ainda' 'algo'
 'altamente' 'amadora' 'amei' 'anterior' 'ao' 'aos' 'apreciam' 'arte'
 'artificiais' 'as' 'assisti' 'assistindo' 'assistir' 'atento' 'atenção'
 

In [9]:
corpus = ["O filme é muito bom, recomendo a todos",
          "O filme é muito bom, mas não recomendo a ninguém",
          "O filme é muito ruim, mas recomendo a todos"]
vectorizer_corpus = TfidfVectorizer()
X_vectorized_corpus = vectorizer_corpus.fit_transform(corpus, y=None)

new_section("Explorando vectorizer.transform()")
subsection("x = 'O filme é bom'")
x = "O filme é bom"
subsection("x_vec = vectorizer_corpus.transform([x])")
x_vec = vectorizer_corpus.transform([x])
print("x_vec.shape :")
print(x_vec.shape)
print("x_vec :")
print(x_vec)
print("type(x_vec) :")
print(type(x_vec))
print("x_vec.todense() :")
print(x_vec.todense())
print("binarizando x_vec.todense() :")
print((x_vec.todense() > 0).astype(int))
sep(sep="*")
subsection("y = 'O filme é muito ruim'")
y = "O filme é muito ruim"
subsection("y_vec = vectorizer_corpus.transform([y])")
y_vec = vectorizer_corpus.transform([y])
print("y_vec.shape :")
print(y_vec.shape)
print("y_vec :")
print(y_vec)
print("type(y_vec) :")
print(type(y_vec))
print("y_vec.todense() :")
print(y_vec.todense())
sep()

============================================ Explorando vectorizer.transform() ============================================

x = 'O filme é bom'------------------------------------------------------------------------------------------------------------------------

x_vec = vectorizer_corpus.transform([x])------------------------------------------------------------------------------------------------------------------------
x_vec.shape :
(1, 9)
x_vec :
  (0, 0)	0.7898069290660905
  (0, 1)	0.6133555370249717
type(x_vec) :
<class 'scipy.sparse._csr.csr_matrix'>
x_vec.todense() :
[[0.78980693 0.61335554 0.         0.         0.         0.
  0.         0.         0.        ]]
binarizando x_vec.todense() :
[[1 1 0 0 0 0 0 0 0]]
************************************************************************************************************************

y = 'O filme é muito ruim'------------------------------------------------------------------------------------------------------------------------

In [10]:
def binarize(x): 
    x_bin = []
    n = len(vectorizer_corpus.get_feature_names_out()) # Número de palavras no vetorizador
    x_bin=np.zeros(n, dtype=int) # Vetor a ser binarizado
    for i in x.indices:
        x_bin[i] = 1 # Binarização, ativamos a palavra contida na sentença
    return x_bin
def binarize2(x): 
    x = (x.todense() > 0).astype(int)
    return x.getA1()
print(binarize(x_vec))
print("*"*120)
print(binarize2(x_vec))
print("*"*120)
print(type(binarize2(x_vec)))


[1 1 0 0 0 0 0 0 0]
************************************************************************************************************************
[1 1 0 0 0 0 0 0 0]
************************************************************************************************************************
<class 'numpy.ndarray'>


In [23]:
a = [1*(1.40546511+1), 1*(1.40546511+1), 2*(1.40546511+1), 1*(1.40546511+1), 2*(1+1), 1*(1.40546511+1), 2*(1+1)]
a_array = np.array(a)
a_22 = a_array@a_array
print()

[2.4054651099999997, 2.4054651099999997, 4.8109302199999995, 2.4054651099999997, 4, 2.4054651099999997, 4, 2.4054651099999997, 2.4054651099999997, 4.8109302199999995, 2.4054651099999997, 4, 2.4054651099999997, 4]


In [12]:
num_samples = 6
def samples(x):
    n = len(vectorizer_corpus.get_feature_names_out())
    x_indices = x.indices
    n_x_words = len(x.indices)    
    sample_set = [np.zeros(n) for i in range(num_samples-1)]
    sample_set.append(binarize2(x))
    weights = x.tocoo().data
    print("weights :")
    print(weights)
    weights_normalized = weights / weights.sum()
    print("weights_normalized :")
    print(weights_normalized)
    for i in range(num_samples-1):
        num_words = np.random.randint(1, n_x_words+1)
        z_line_indices = np.random.choice(x_indices, size=num_words, p=weights_normalized, replace=False)
        sample_set[i][z_line_indices] = 1
    return sample_set
print(y_vec)
sampless = samples(y_vec)
for sample in sampless:
    print(sample)

  (0, 1)	0.4532946552278861
  (0, 3)	0.4532946552278861
  (0, 7)	0.7674945674619879
weights :
[0.45329466 0.45329466 0.76749457]
weights_normalized :
[0.27077177 0.27077177 0.45845646]
[0. 1. 0. 1. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 0. 0. 1. 0.]
[0. 0. 0. 1. 0. 0. 0. 1. 0.]
[0. 0. 0. 1. 0. 0. 0. 0. 0.]
[0. 1. 0. 1. 0. 0. 0. 1. 0.]
[0 1 0 1 0 0 0 1 0]


In [13]:
print(X_vectorized_corpus)
lista_de_palavras = vectorizer_corpus.get_feature_names_out() 
print(lista_de_palavras)
super_frase = " ".join(lista_de_palavras)
print(super_frase)
super_frase = vectorizer_corpus.transform([super_frase])
print(super_frase)
print(super_frase.tocoo().data)   
print(super_frase.indices)

  (0, 1)	0.39789669894933666
  (0, 3)	0.39789669894933666
  (0, 0)	0.5123644459248041
  (0, 6)	0.39789669894933666
  (0, 8)	0.5123644459248041
  (1, 1)	0.2880786492345198
  (1, 3)	0.2880786492345198
  (1, 0)	0.3709537120754161
  (1, 6)	0.2880786492345198
  (1, 2)	0.3709537120754161
  (1, 5)	0.4877595527309447
  (1, 4)	0.4877595527309447
  (2, 1)	0.3299953074645687
  (2, 3)	0.3299953074645687
  (2, 6)	0.3299953074645687
  (2, 8)	0.4249290414153384
  (2, 2)	0.4249290414153384
  (2, 7)	0.5587306244316468
['bom' 'filme' 'mas' 'muito' 'ninguém' 'não' 'recomendo' 'ruim' 'todos']
bom filme mas muito ninguém não recomendo ruim todos
  (0, 0)	0.3162910421180881
  (0, 1)	0.2456282096992013
  (0, 2)	0.3162910421180881
  (0, 3)	0.2456282096992013
  (0, 4)	0.4158847107181897
  (0, 5)	0.4158847107181897
  (0, 6)	0.2456282096992013
  (0, 7)	0.4158847107181897
  (0, 8)	0.3162910421180881
[0.31629104 0.24562821 0.31629104 0.24562821 0.41588471 0.41588471
 0.24562821 0.41588471 0.31629104]
[0 1 2 3 4 5 

In [14]:
def sentences_samples(z_line):
    indices = np.where(z_line == 1)[0]
    z=" ".join([vectorizer_corpus.get_feature_names_out()[indice] for indice in indices])
    return vectorizer_corpus.transform([z])
for i in sampless:
    print(sentences_samples(i))
    print("*"*120)

  (0, 1)	0.4532946552278861
  (0, 3)	0.4532946552278861
  (0, 7)	0.7674945674619879
************************************************************************************************************************
  (0, 7)	1.0
************************************************************************************************************************
  (0, 3)	0.5085423203783267
  (0, 7)	0.8610369959439764
************************************************************************************************************************
  (0, 3)	1.0
************************************************************************************************************************
  (0, 1)	0.4532946552278861
  (0, 3)	0.4532946552278861
  (0, 7)	0.7674945674619879
************************************************************************************************************************
  (0, 1)	0.4532946552278861
  (0, 3)	0.4532946552278861
  (0, 7)	0.7674945674619879
*************************************************************

In [15]:
new_section("Teste de cosine_similarity")
def cosine_similarity_my(x, z):
    x = x.todense()
    z = z.todense()
    dot_product = x.dot(z.T)
    norm_x = np.linalg.norm(x)
    norm_z = np.linalg.norm(z)
    return (dot_product / (norm_x * norm_z)).getA1()[0]
def kernel(x, z):
    distance = cosine_similarity_my(x, z) # Similaridade de cosseno
    weight = np.sqrt(np.exp(-(distance**2) / (0.2**2)))  # Kernel exponencial
    return weight
x_vec = vectorizer_corpus.transform(["não recomendo muito"])
y_vec = vectorizer_corpus.transform(["nignuém mas filme bom, recomendo muito"])
print(x_vec)
print("----------")
print(y_vec)
subsection("cosine_similarity(x_vec, y_vec)")
print(cosine_similarity(x_vec, y_vec))
subsection("cosine_similarity_my(X_vectorized[instance_index_neg], X_vectorized[instance_index_pos])")
print(cosine_similarity_my(x_vec,y_vec))
subsection("kernel(x_vec, y_vec)")
print(kernel(x_vec, y_vec))

sep()


=============================================== Teste de cosine_similarity ===============================================
  (0, 3)	0.4532946552278861
  (0, 5)	0.7674945674619879
  (0, 6)	0.4532946552278861
----------
  (0, 0)	0.512364445924804
  (0, 1)	0.3978966989493366
  (0, 2)	0.512364445924804
  (0, 3)	0.3978966989493366
  (0, 6)	0.3978966989493366

cosine_similarity(x_vec, y_vec)------------------------------------------------------------------------------------------------------------------------
[[0.36072889]]

cosine_similarity_my(X_vectorized[instance_index_neg], X_vectorized[instance_index_pos])------------------------------------------------------------------------------------------------------------------------
0.3607288939331071

kernel(x_vec, y_vec)------------------------------------------------------------------------------------------------------------------------
0.19660341787972543


In [16]:
# Lime para explicação de texto    
class LimeExplainerSentences:
    def __init__(self, sigma=25**2, num_samples=15000, K=6, alpha=0.1**(16), p=16,vectorizer=None, model=None, seed=42, generate="opt"):
        self.sigma = sigma # Parâmetro do kernel exponencial
        self.num_samples = num_samples # Número de amostras geradas 
        self.K = K # Número de palavras importantes
        self.alpha = alpha # Parâmetro de regularização Lasso
        self.model = model # Modelo de classificação
        self.vectorizer = vectorizer # Vetorizador
        self.generate = generate # Tipo de amostragem
        self.p = p # Precisão de impressão
        np.random.seed(seed) # Semente aleatória
        np.set_printoptions(precision=self.p) # Precisão de impressão
        

    # Binarizar vetor de palavras
    def binarize(self, x): 
        x = (x.todense() > 0).astype(int) # Binariza o vetor
        return x.getA1() # Retorna o vetor em formato de array

    def cossine_similarity(self, x, z):
        x = x.todense() # Transforma o vetor em uma matriz densa
        z = z.todense()
        dot_product = x.dot(z.T)
        norm_x = np.linalg.norm(x)
        norm_z = np.linalg.norm(z)
        return (dot_product / (norm_x * norm_z)).getA1()[0]
    
    # Define a função de kernel 
    def kernel(self, x, z):
        distance = self.cossine_similarity(x, z) # Similaridade de cosseno
        weight = np.sqrt(np.exp(-(distance**2) / (self.sigma**2)))  # Kernel exponencial
        return weight
    
    
    def samples_simples(self, x):
        n = len(self.vectorizer.get_feature_names_out()) # Número de palavras no vetorizador
        x_indices = x.indices # Índices das palavras na sentença
        n_x_words = len(x.indices) # Número de palavras na sentença   
        sample_set = [np.zeros(n) for i in range(self.num_samples-1)] # Conjunto de amostras
        sample_set.append(self.binarize(x)) # Adiciona a sentença original binarizada
        Z_line_indices = [] # Índices das palavras ativadas
        for i in range(self.num_samples-1): # Gerar amostras aleatórias
            num_words = np.random.randint(1, n_x_words+1)  # Número de palavras na amostra
            # Escolhe aleatoriamente as palavras da sentença original
            # size = número de palavras na amostra
            # replace = False, não permite repetição
            z_line_indices = np.random.choice(x_indices, size=num_words, replace=False)
            # Ativa as palavras escolhidas
            sample_set[i][z_line_indices] = 1
            Z_line_indices.append(z_line_indices)
        return sample_set, z_line_indices

    # Gera dados ao redor de x_line relevancia das palavras
    def samples_opt(self, x):
        n = len(self.vectorizer.get_feature_names_out()) # Número de palavras no vetorizador
        x_indices = x.indices # Índices das palavras na sentença
        n_x_words = len(x.indices) # Número de palavras na sentença   
        sample_set = [np.zeros(n) for i in range(self.num_samples-1)] # Conjunto de amostras
        sample_set.append(self.binarize(x)) # Adiciona a sentença original binarizada
        weights = x.tocoo().data # Pesos das palavras
        weights_normalized = weights / weights.sum() # Normaliza os pesos
        Z_line_indices = [] # Índices das palavras ativadas
        for i in range(self.num_samples-1): # Gerar amostras aleatórias
            num_words = np.random.randint(1, n_x_words+1)  # Número de palavras na amostra
            # Escolhe aleatoriamente as palavras da sentença original
            # size = número de palavras na amostra
            # p = pesos normalizados, a probabilidade de escolher cada palavra
            # replace = False, não permite repetição
            z_line_indices = np.random.choice(x_indices, size=num_words, p=weights_normalized, replace=False)
            # Ativa as palavras escolhidas
            sample_set[i][z_line_indices] = 1
            Z_line_indices.append(z_line_indices)
        return sample_set, z_line_indices
    
    
    # Transforma um vetor em uma frase
    def sentences_samples(self, Z_line):
        for z_line in Z_line:
            z=" ".join([self.vectorizer.get_feature_names_out()[indice] for indice in Z_line])
        return self.vectorizer.transform([z])

    # Define o vetor de pesos
    def LIME(self, x):
        if self.generate == "opt":
            Z_line = self.samples_opt
        else:
            Z_line = self.samples_simples
        Z=[]
        for i in range(len(Z_line)):
            Z.append(self.sentences_samples(Z_line[i]))
        Z_pred = np.array([self.model.predict(z)[0] for z in Z])  
        pi_x = np.array([self.kernel(x, z)[0][0] for z in Z]) 
        lasso = Lasso(alpha=self.alpha)
        lasso.fit(Z_line, Z_pred, sample_weight=pi_x)
        w = lasso.coef_
        return w 

    # Gerar explicação
    def explain_instance(self, x):
        w = self.LIME(x)
        abs_valores = np.abs(w)
        indices = np.argsort(abs_valores)[::-1][:self.K]
        print("Palavras importantes:")
        for i in indices:
            print(f"{self.vectorizer.get_feature_names_out()[i]}: {w[i]}")    
      


In [17]:
class SubmodularPick:
    def __init__(self,X_n_vec, X, B,lime=None, vectorizer=None):
        self.X_n_vec = X_n_vec
        self.X = X
        self.B = B
        self.lime = lime
        self.vectorizer = vectorizer

    def explain_instances(self, X):
        W = [self.lime.LIME(x) for x in X] 
        return np.array(W)      
        
    def importancia(self, W):
        I = np.zeros(W.shape[1])  
        for j in range(W.shape[1]):
            soma = np.sum(np.abs(W[:, j]))
            I[j] = np.sqrt(soma)
        return I

    def cobertura(self, V, W, I):
        c_value = 0  
        for j in range(W.shape[1]):
            if any(W[i, j] > 0 for i in V):  # Se a característica j é relevante
                c_value += I[j]
        return c_value

    def guloso(self, X, B):
        W = self.explain_instances(X) 
        I = self.importancia(W)
        nao_selecionados = list(range(W.shape[0]))  # Lista de índices não selecionados
        V = []  # Conjunto de características selecionadas
        itens = 0  # Número de elementos selecionados
        c_value = 0  # Valor de c

        while itens < B and nao_selecionados:
            best_gain = -np.inf  # Valor de ganho máximo
            best_item = None  # Índice do melhor item 

            for item in nao_selecionados:  # Itera sobre os itens não selecionados
                lista_temp = V + [item]
                gain = self.cobertura(lista_temp, W, I) - self.cobertura(V, W, I)
                if gain > best_gain:
                    best_gain = gain  # Atualiza o valor de ganho máximo
                    best_item = item  # Atualiza o melhor item

            if best_item is not None:
                V.append(best_item)  # Adiciona o melhor item ao conjunto
                nao_selecionados.remove(best_item)  # Remove o melhor item dos não selecionados
                itens += 1
                c_value += best_gain

        return V
    
    def explain_model(self, X):
        V = self.guloso(X, self.B)
        print("Melhor conjunto de explicação: ", V)
        for i in V:
            print(f"{self.X_n_vec[i]}")
        print("Fim da explicação")  